# Structuring an API call 


In [ ]:
from openai import OpenAI
import os 
from IPython.display import Markdown

In [ ]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages =[{
        'role': 'user', 
        'content': 'Who developed chatgpt?'
    }]
)

print(response.choices[0].message.content)

# Challenges on a production environment

## Error Handling 
- Display useful error messages
- Alternatives for when the service is unavailable

## Moderation and Safety 
- Control unwanted inputs
- Minimizing the risk of data leaks

## Testing and validation 
- Checking for responses that are out of topic
- Testing for inconsisten behavior

## Communication with External Systems 
- Calling external functions and APIs
- Optimizing response times

# Specific Output Format


In [ ]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages =[{
        'role': 'user', 
        'content': 'Give me the best 3 novels of all time in json format'
    }]
)

Markdown(response.choices[0].message.content)

In [ ]:
# Create the request
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
   {"role": "user", "content": "Give me the 3 best graphic novels of all time in json format"}
  ],
  # Specify the response format
  response_format={"type": "json_object"}
)

# Print the response
print(response.choices[0].message.content)

# Handling errors


In [ ]:
response = client.chat.completions.create(
    model='text-davinci-001', 
    messages=[
        {
            "role": "user",
            "content": "give me the 3 best post rock albums of all time in json format"
        }
   ],
    response_format='json_object'
)

Markdown(response.choices[0].content.message)

## Connection Errors
Common expectable errors on either server or client side are **InternalServerError**, **APIConnectionError** or **APITimeourError**

In these cases: 
- Check connection configuration
- Reach out to support

## Resource Limits Errors 
**ConflictError** and **RateLimitError** 

- Check limit restrictions
- Ensure requests are within limits

## Authentication errors 
Wrong, expired or revoked api keys

## Bad Request errors
Check the expected request documentation for missing keys or typos.


# Handling Errors

In [ ]:
try: 
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages= [{
            'role': 'user',
            'content': 'list five data science professions'
        }]
    )
except openai.AuthenticationError as e:
    print(f'OpenAI failed to authenticate: {e}')
    pass
except openai.RateLimitError as e:
    print(f'OpenAI API request exceeded rate limit: {e}')
    pass
except Exception as e: 
    print(f'Unable to generate a response. Exception: {e}')
    pass 



# RateErrorLimit 

The RateLimitError could be caused by: 
- Too many requests in a short period of time
- Too many tokens in the request

## Solutions 

### Retry (short wait between the requests)
Retry can be achieved with the python's tenacity library

In [ ]:
from tenacity import (retry, stop_after_attempt, wait_random_exponential)

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(2))
def get_response():
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages= [{
            'role': 'user',
            'content': 'list five data science professions'
        }]
    )
    print(response.choices[0].message.content)

get_response()

### Batching (Processing multiple messages in one request)

In [ ]:
bands = ['Pearl Jam', 'Radiohead', 'Sigur Ros']

message = [
    {
        'role': 'system', 
        'content': '''You are given a series of rock bands and are asked to return 
        their best album along with the publication year. Provide each answer in the 
        response as a separate content''',
    }
]
[message.append({'role': 'user', 'content': band}) for band in bands]
print(message)

In [ ]:
response = client.chat.completions.create(
    model='gpt-4o-mini', 
    messages=message
)

In [ ]:
print(response.choices[0].message.content)

### Reducing the amount of tokens


In [ ]:
import tiktoken 

encoding = tiktoken.encoding_for_model('gpt-4o-mini')
prompt = 'tokens can be full words or groups of characters commonly grouped together: tokenization'

encoded = encoding.encode(prompt)
num_tokens = len(encoded)
print(f"Num tokens: {num_tokens}")
print(f"encoded: {encoded[:10]}")


In [ ]:
input_message = {"role": "user", "content": "I'd like to buy a shirt and a jacket. Can you suggest two color pairings for these items?"}

# Use tiktoken to create the encoding for your model
encoding = tiktoken.encoding_for_model('gpt-4o-mini')
# Check for the number of tokens
num_tokens = len(encoding.encode(input_message['content']))

# Run the chat completions function and print the response
if num_tokens <= 100:
    response = client.chat.completions.create(model="gpt-4o-mini", messages=[input_message])
    print(response.choices[0].message.content)
else:
    print("Message exceeds token limit")

# Defining Function Calling 

